In [36]:
from env import conn
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import datetime as dt


In [22]:
def return_df(query):
    cs = conn.cursor()
    query = query
    result = cs.execute(query)
    df = result.fetch_pandas_all()
    cs.close()
    return df


# Investigating launch for Stonegate

In [25]:
tables = return_df(
    """
    SELECT TABLE_SCHEMA, TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE'
order by table_schema, table_name;    
    """
        )
tables


,TABLE_SCHEMA,TABLE_NAME
0,JIRA,JIRA__SPRINT__SPRINT
1,JIRA,JIRA__TEAM__SPRINT
2,LOYALTY_CARDS,LC201__LOYALTY_CARD_JOURNEY_FUNNEL__USER_LEVEL...
3,LOYALTY_CARDS,LC__ERRORS__DAILY_STATUS_ROLLUP_USER_LEVEL
4,LOYALTY_CARDS,LC__LINKS_JOINS__DAILY_RETAILER
5,LOYALTY_CARDS,LC__LINKS_JOINS__DAILY_RETAILER_CHANNEL
6,LOYALTY_CARDS,LC__LINKS_JOINS__DAILY_RETAILER_CHANNEL__FORECAST
7,LOYALTY_CARDS,LC__LINKS_JOINS__MONTHLY_RETAILER
8,LOYALTY_CARDS,LC__LINKS_JOINS__MONTHLY_RETAILER_CHANNEL
9,LOYALTY_CARDS,LC__LINKS_JOINS__MONTHLY_RETAILER_CHANNEL__GROWTH


Generating dataframnes for analysis

In [30]:
lc_daily_metrics = return_df(
    """select * from METRICS.LOYALTY_CARDS.LC__LINKS_JOINS__DAILY_RETAILER
        where loyalty_plan_company = 'Stonegate Group'
        order by date;    
    """
        )

txn_daily_metrics = return_df(
    """select * from METRICS.TRANSACTIONS.TRANS__TRANS__DAILY_RETAILER
        where loyalty_plan_company = 'Stonegate Group'
         order by date;    
    """
        )

user_level_txns = return_df(
    """select * from PROD.BINK_SECURE.FACT_TRANSACTION
        where loyalty_plan_company = 'Stonegate Group'
         order by event_date_time;    
    """
        )


##### Categorising users based on txns

In [37]:
user_level_txns.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139554 entries, 0 to 139553
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   EVENT_ID               139554 non-null  int32         
 1   EVENT_DATE_TIME        139554 non-null  datetime64[ns]
 2   USER_ID                139554 non-null  object        
 3   EXTERNAL_USER_REF      139554 non-null  int64         
 4   CHANNEL                139554 non-null  object        
 5   BRAND                  139554 non-null  object        
 6   TRANSACTION_ID         139554 non-null  object        
 7   PROVIDER_SLUG          139554 non-null  object        
 8   FEED_TYPE              0 non-null       object        
 9   DUPLICATE_TRANSACTION  139554 non-null  bool          
 10  LOYALTY_PLAN_NAME      139554 non-null  object        
 11  LOYALTY_PLAN_COMPANY   139554 non-null  object        
 12  TRANSACTION_DATE       139554 non-null  date

In [35]:
#hash external_user_ref
user_level_txns["EXTERNAL_USER_REF"] = user_level_txns["EXTERNAL_USER_REF"].apply(hash)

user_level_txns_agg = user_level_txns.groupby(by="EXTERNAL_USER_REF", ).agg({'TRANSACTION_ID':'count', 
                         'SPEND_AMOUNT':['sum', 'mean', 'min', 'max', lambda x: x.max() - x.min()],
                         'EVENT_DATE_TIME':['min', 'max', lambda x: x.max() - x.min()]})
user_level_txns_agg


TRANSACTION_ID SPEND_AMOUNT                           \
                              count          sum       mean    min    max   
EXTERNAL_USER_REF                                                           
-9223124276114859340              9        67.11   7.456667   3.53  11.73   
-9221964618865320895              2        17.17   8.585000   7.50   9.67   
-9221416666281670135              5        42.28   8.456000   4.40  16.05   
-9220696902008348704              1        16.98  16.980000  16.98  16.98   
-9220614847841984224              2        16.35   8.175000   4.40  11.95   
...                             ...          ...        ...    ...    ...   
 9219299568670200754             13        74.60   5.738462   3.30  14.50   
 9220700972050154462              5        20.70   4.140000   3.45   6.90   
 9221600873427300712             48       253.98   5.291250   1.25  19.99   
 9221873185518117586              4        24.50   6.125000   1.35  10.05   
 9222640852897801657              4        26.40   6.600000   5.15  10.95   

                                           EVENT_DATE_TIME  \
                     <lambda_0>                        min   
EXTERNAL_USER_REF                                            
-9223124276114859340       8.20 2023-11-08 07:32:43.959885   
-9221964618865320895       2.17 2023-11-22 23:16:05.683128   
-9221416666281670135      11.65 2023-11-28 18:12:02.238286   
-9220696902008348704       0.00 2023-11-13 20:26:14.667908   
-9220614847841984224       7.55 2023-11-24 10:56:39.881425   
...                         ...                        ...   
 9219299568670200754      11.20 2023-11-19 06:28:11.824675   
 9220700972050154462       3.45 2023-11-19 16:08:07.371160   
 9221600873427300712      18.74 2023-11-08 08:16:44.003111   
 9221873185518117586       8.70 2023-11-28 11:22:21.793286   
 9222640852897801657       5.80 2023-11-22 22:25:23.624667   

                                                 
                                            max  
EXTERNAL_USER_REF                                
-9223124276114859340 2023-11-23 09:31:39.376845  
-9221964618865320895 2023-11-23 07:22:54.629212  
-9221416666281670135 2023-11-28 18:21:05.728195  
-9220696902008348704 2023-11-13 20:26:14.667908  
-9220614847841984224 2023-11-24 10:56:40.053711  
...                                         ...  
 9219299568670200754 2023-11-28 11:43:43.289453  
 9220700972050154462 2023-11-26 07:19:01.652190  
 9221600873427300712 2023-11-29 08:45:54.589925  
 9221873185518117586 2023-11-29 11:13:03.114711  
 9222640852897801657 2023-11-29 09:14:55.659621  

[22920 rows x 8 columns]

In [ ]:
user
